In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import gradio as gr  # Import Gradio

# Load data
data = pd.read_excel("DATA.xlsx")

# Prepare the data
data['PF Average'] = data[['Y-PF']].mean(axis=1)
filtered_data = data[(data['PF Average'] >= 0.7) & (data['PF Average'] <= 9.5)]

# Predefined features (X) and target (y)
X_columns = ['R PHASE V', 'Y PHASE V', 'B PHASE V', 'R CURRENT', 'Y CURRENT', 'B CURRENT']
X = filtered_data[X_columns]
y = filtered_data['PF Average']

# Train the model with RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

# Function to make predictions from an uploaded Excel file
def predict_from_excel(file):
    # Read the uploaded Excel file
    input_data = pd.read_excel(file)

    # Check if the necessary columns are present in the uploaded file
    required_columns = ['R PHASE V', 'Y PHASE V', 'B PHASE V', 'R CURRENT', 'Y CURRENT', 'B CURRENT']
    if not all(col in input_data.columns for col in required_columns):
        return "Error: Excel file must contain the following columns: R PHASE V, Y PHASE V, B PHASE V, R CURRENT, Y CURRENT, B CURRENT"

    # Filter the data to get only necessary columns for prediction
    X_new = input_data[required_columns]
    
    # Predict the PF Average for each row in the uploaded Excel file
    predictions = model.predict(X_new)
    
    # Add predictions to the DataFrame
    input_data['Predicted PF Average'] = predictions

    # Calculate the average of the predicted PF Average
    average_prediction = predictions.mean()

    # Return the updated DataFrame with predictions and the average
    return input_data, f"Average Predicted PF Average: {average_prediction:.4f}"

# Creating the Gradio interface
interface = gr.Interface(
    fn=predict_from_excel,
    inputs=gr.File(label="Upload Excel File"),  # File input for uploading Excel file
    outputs=[
        gr.Dataframe(label="Predicted PF Average"),  # Output the DataFrame with predictions
        gr.Textbox(label="Average Predicted PF Average")  # Output the average prediction
    ],
    title="PF Average Prediction from Excel",
    description="Upload an Excel file containing R PHASE V, Y PHASE V, B PHASE V, R CURRENT, Y CURRENT, and B CURRENT to predict the PF Average using a random forest regression model and get the average of predictions."
)

# Launch the interface
interface.launch(share=True)
